Test Loading Workflow Metadata.ipynb

As an interface between the analysis pipeline and the Napari GUI environment, using the workflow assistant. 

The steps of the image analysis pipeline are exported as a 
.py file and/or yaml file, containing the order and parameters for the steps used. 

This notebook will import that info and turn it into a processing pipeline. 

Ultimately, the goal is to have a user generated metadatafile for each experimental replicate, with parameters adapted to segmentaing cells as best possible from a sample of cropped regions from multiple image planes. 

These replicate-specific parameters will be saved into a folder, and loaded as a part of this analysis pipeline.


In [1]:
# Example filepaths for testing

workflow_dir = 'M://Brain_Registration/napari-clesperanto/'

workflow_py = 'test_workflow.py'
workflow_yaml = 'test_workflow.yaml'




In [5]:
import yaml

def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            data = yaml.safe_load(stream)

            print(data)
            return data
        except yaml.YAMLError as exc:
            print(exc)


In [6]:
def Workflow_constructor(loader, node):
    value = loader.construct_mapping(node)
    # Instantiate your Workflow object based on the loaded values
    obj = napari_workflows._workflow.Workflow(value)
    return obj

# Register the constructor for the Workflow object
yaml.SafeLoader.add_constructor(u'!!python/object:napari_workflows._workflow.Workflow', Workflow_constructor)

# Now, you should be able to load your yaml file using the `safe_load()` function:
data = read_yaml_file(workflow_dir+workflow_yaml)
display(data)


could not determine a constructor for the tag 'tag:yaml.org,2002:python/object:napari_workflows._workflow.Workflow'
  in "M://Brain_Registration/napari-clesperanto/test_workflow.yaml", line 1, column 1


None

In [14]:
import ruamel.yaml

def parse_workflow_yaml(file_path):
    # Create YAML loader
    yaml = ruamel.yaml.YAML(typ='safe')  # 'safe' means it won't load any Python objects

    # Load YAML file
    with open(file_path) as file:
        data = yaml.load(file)

    # Extract tasks
    tasks = data.get('_tasks', {})

    # Initialize empty dict for parsed tasks
    parsed_tasks = {}

    # Iterate over tasks
    for task_name, task_data in tasks.items():
        # Skip if not a tuple (which indicates a function call)
        if not isinstance(task_data, list):
            continue

        # Extract function name and parameters
        function_name = task_data[0].split('.')[-1]  # Get last part of function name
        parameters = task_data[1:]

        # Convert 'null' to None
        parameters = [None if param == 'null' else param for param in parameters]

        # Save parsed task
        parsed_tasks[task_name] = {
            'function_name': function_name,
            'parameters': parameters,
        }

    return parsed_tasks

data = parse_workflow_yaml(workflow_dir+workflow_yaml)
display(data)


ModuleNotFoundError: No module named 'ruamel'

In [15]:
import yaml

def parse_workflow_yaml(file_path):
    tasks = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        task_name = None
        for line in lines:
            if "!!python" not in line:
                if ":" in line and line[0] != " ":
                    task_name = line.split(":")[0].strip()
                    tasks[task_name] = {'function_name': '', 'parameters': []}
                elif task_name:
                    if "!!python/name" in line:
                        tasks[task_name]['function_name'] = line.split("''")[0].split(".")[-1].strip()
                    else:
                        param = line.strip().strip('-').strip()
                        if param == 'null':
                            param = None
                        elif param.replace('.','').isdigit():
                            param = float(param)
                        tasks[task_name]['parameters'].append(param)
    return tasks

data = parse_workflow_yaml(workflow_dir+workflow_yaml)
display(data)

{'_tasks': {'function_name': '',
  'parameters': ['Result of top_hat_box (clesperanto)',
   None,
   1.0,
   1.0,
   0.0,
   'Result of top_hat_box (clesperanto)',
   '356850_415210_044800_cropped_0',
   None,
   2.0,
   2.0,
   0.0,
   'Result of threshold_otsu (clesperanto)',
   None,
   2.0,
   2.0]}}

In [17]:
def parse_workflow_yaml(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # initialize the list of commands and the dictionary of variables
    commands = []
    variables = {}

    # iterate over the lines in the file
    for line in lines:
        if line.startswith("  "):  # if the line is indented, it's a task
            if "!!python/tuple" in line:  # if it's a command
                # extract the command name and parameters
                line = line.replace("!!python/tuple", "")
                line = line.replace("!!python/name:", "")
                line = line.strip()
                task_name, rest = line.split(":")
                rest = rest.split("-")
                command_name = rest[1].strip()
                params = [param.strip() for param in rest[2:]]

                # replace parameter names with corresponding variable names
                for i in range(len(params)):
                    if params[i] in variables:
                        params[i] = variables[params[i]]

                # add the command to the list of commands
                commands.append({
                    "command_name": command_name,
                    "params": params
                })

                # add the output variable to the dictionary of variables
                variables[task_name] = command_name


    return commands

data = parse_workflow_yaml(workflow_dir+workflow_yaml)
display(data)

IndexError: list index out of range

In [11]:

def write_python_script(data, script_filename):
    with open(script_filename, 'w') as script_file:
        script_file.write("import pyclesperanto_prototype as cle\n")
        script_file.write("from skimage.io import imread\n")
        script_file.write("from napari import Viewer\n\n")
        script_file.write("def pipeline(input_image):\n")
        script_file.write("\tviewer = Viewer()\n\n")

        for key, value in data.items():

            print(key, value)
            if isinstance(value, list) and len(value) > 0 and "pyclesperanto_prototype" in value[0]:
                # write the name of the function and open bracket
                script_file.write("\t" + key + " = " + value[0].split(".")[-1] + "(")

                # write the parameters of the function
                for param in value[1:]:
                    if isinstance(param, str):
                        if param.isnumeric():
                            script_file.write(param + ", ")
                        else:
                            script_file.write("'" + param + "', ")
                    elif param is None:
                        script_file.write("None, ")
                    else:
                        script_file.write(str(param) + ", ")
                
                # remove last ", " and close bracket
                script_file.seek(script_file.tell() - 2, os.SEEK_SET)
                script_file.write(")\n")
                
                # add image to viewer
                script_file.write("\tviewer.add_image(" + key + ", name='" + key + "')\n\n")

        script_file.write("\treturn viewer")

In [12]:
# Write the Python script
write_python_script(data, "generated_pipeline_script.py")

_tasks {'function_name': '', 'parameters': ['Result of top_hat_box (clesperanto)', None, 1.0, 1.0, 0.0, 'Result of top_hat_box (clesperanto)', '356850_415210_044800_cropped_0', None, 2.0, 2.0, 0.0, 'Result of threshold_otsu (clesperanto)', None, 2.0, 2.0]}
